# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP. 

First we import the necessary packages and create the global variables.

In [25]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib
import seaborn as sns
import pandas as pd
import time

import CMAPSAuxFunctions
#import plottingTools
#from datetime import datetime
#from sklearn.covariance import EllipticEnvelope
#from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#from sklearn.dummy import DummyClassifier
#from sklearn.model_selection import train_test_split, cross_validate
#from sklearn.neural_network import MLPClassifier
#from mpl_toolkits.mplot3d import Axes3D
#from dataManagement import DataManagerDamadics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from IPython.display import display, HTML
%matplotlib notebook

global constRUL

constRUL = 125
time_window = 30
rul_vector = None

# Retrieve and Reshape data

Get the data from the text files, store it in a Pandas Dataframe and reshape it as appropiately.

In [26]:
data_file_train = '../CMAPSSData/train_FD001.txt'
data_file_test = '../CMAPSSData/test_FD001.txt'

#min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
standardScaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

nFeatures = len(selected_features)

#Get the X and y matrices with the specified time window
X_train, y_train = CMAPSAuxFunctions.retrieve_and_reshape_data(data_file_train, selected_features, time_window, 'train')
X_test, _ = CMAPSAuxFunctions.retrieve_and_reshape_data(data_file_test, selected_features, time_window, 'test')
y_test = np.loadtxt("../CMAPSSData/RUL_FD001.txt")
y_test = np.array([x if x < constRUL else constRUL for x in y_test])
y_test = np.reshape(y_test, (y_test.shape[0], 1))

X_train, y_train, X_test, y_test = CMAPSAuxFunctions.get_X_y_from_Dataset('1', '../CMAPSSData/', constRUL, time_window)
X_train = np.reshape(X_train, newshape=(X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
X_test = np.reshape(X_test, newshape=(X_test.shape[0], X_test.shape[1]*X_test.shape[2]))
y_train = np.reshape(y_train, newshape=(y_train.shape[0], 1))
y_test = np.reshape(y_test, newshape=(y_test.shape[0], 1))


#Standardize the data
#X_train = min_max_scaler.fit_transform(X_train)
#X_test = min_max_scaler.fit_transform(X_test)

In [27]:
print("Training data")
print(X_train.shape)
print(y_train.shape)
print("Testing data")
print(X_test.shape)
print(y_test.shape)
print("Training data")
print(X_train[-5:,:])
print(y_train[-5:,:])
print("Testing data")
print(X_test[-5:,:])
print(y_test[-5:,:])

Training data
(17731, 420)
(17731, 1)
Testing data
(100, 420)
(100, 1)
Training data
[[ 0.42168675  0.12579028  0.17049291 ...  0.5        -0.45736434
  -0.78099972]
 [ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.75193798
  -0.26760563]
 [-0.01204819 -0.02419882  0.47467927 ...  0.66666667 -0.53488372
  -0.89201878]
 [-0.09638554 -0.02419882  0.53376097 ...  0.16666667 -0.76744186
  -0.53106877]
 [ 0.10843373 -0.01809462  0.29068197 ...  0.33333333 -0.64341085
  -0.56365645]]
[[4]
 [3]
 [2]
 [1]
 [0]]
Testing data
[[-0.55421687 -0.61107478 -0.55232951 ... -0.5         0.27131783
   0.56420878]
 [-0.02409639 -0.30978853 -0.32343011 ...  0.16666667 -0.27131783
   0.10770505]
 [-0.15662651 -0.05777196 -0.29642134 ...  0.         -0.03875969
   0.28859431]
 [-0.40963855 -0.19032047 -0.56617151 ... -0.5         0.25581395
   0.28500414]
 [-0.30722892 -0.11881404 -0.1144497  ...  0.33333333 -0.13178295
  -0.1955261 ]]
[[125]
 [ 82]
 [ 59]
 [117]
 [ 20]]


# Keras model

We will use a very simple ANN for this example. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [28]:
def RULmodel(input_shape):
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(500, input_dim=input_shape, activation='tanh', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='tanh', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.5))
    #model.add(Dense(100, activation='tanh', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model

# Fit the keras model
Fit the Keras model to the data and determine its performance.

In [29]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
opt = Adam(lr=0, beta_1=0.5)

#Create the model
modelRUL = RULmodel(X_train.shape[1])

#Compile the model.
modelRUL.compile(optimizer = opt, loss = "mean_squared_error", metrics = ["mse"])

startTime = time.clock()
#Train the model.
modelRUL.fit(x = X_train, y = y_train, epochs = 250, batch_size = 512, callbacks=[lrate])  
endTime = time.clock()

420
Epoch 1/250
17731/17731 [==============================] - 0s 13us/step - loss: 6335.1290 - mean_squared_error: 6335.1290
Epoch 2/250
17731/17731 [==============================] - 0s 9us/step - loss: 5675.1038 - mean_squared_error: 5675.1038
Epoch 3/250
17731/17731 [==============================] - 0s 9us/step - loss: 5244.6205 - mean_squared_error: 5244.6205
Epoch 4/250
17731/17731 [==============================] - 0s 9us/step - loss: 4851.1704 - mean_squared_error: 4851.1704
Epoch 5/250
17731/17731 [==============================] - 0s 9us/step - loss: 4490.4882 - mean_squared_error: 4490.4882
Epoch 6/250
17731/17731 [==============================] - 0s 9us/step - loss: 4151.0204 - mean_squared_error: 4151.0204
Epoch 7/250
17731/17731 [==============================] - 0s 9us/step - loss: 3839.1931 - mean_squared_error: 3839.1931
Epoch 8/250
17731/17731 [==============================] - 0s 9us/step - loss: 3545.9781 - mean_squared_error: 3545.9781
Epoch 9/250
17731/17731 [==

Epoch 136/250
17731/17731 [==============================] - 0s 8us/step - loss: 256.2809 - mean_squared_error: 256.2809
Epoch 137/250
17731/17731 [==============================] - 0s 8us/step - loss: 257.7277 - mean_squared_error: 257.7277
Epoch 138/250
17731/17731 [==============================] - 0s 10us/step - loss: 251.2366 - mean_squared_error: 251.2366
Epoch 139/250
17731/17731 [==============================] - 0s 10us/step - loss: 256.5770 - mean_squared_error: 256.5770
Epoch 140/250
17731/17731 [==============================] - 0s 9us/step - loss: 256.8790 - mean_squared_error: 256.8790
Epoch 141/250
17731/17731 [==============================] - 0s 9us/step - loss: 256.9468 - mean_squared_error: 256.9468
Epoch 142/250
17731/17731 [==============================] - 0s 9us/step - loss: 254.4369 - mean_squared_error: 254.4369
Epoch 143/250
17731/17731 [==============================] - 0s 9us/step - loss: 256.4954 - mean_squared_error: 256.4954
Epoch 144/250
17731/17731 [===

In [11]:
#Evaluate the model
score = modelRUL.evaluate(x = X_test, y = y_test)
y_pred = modelRUL.predict(X_test)
healtScore = CMAPSAuxFunctions.compute_health_score(y_test, y_pred)

print("Root Square Mean Error score: {}".format(np.sqrt(score[0])))
print("Health score: {}".format(healtScore))
print("Elapsed time: {}".format(endTime - startTime))

100/100 [==============================] - 0s 290us/step
Root Square Mean Error score: 14.195613320521813
Health score: [507.31751693]
Elapsed time: 41.29436907207547


# CNN Model
Fit the Keras model to the data using a CNN and determine its performance.

# Initialization

In [13]:
import os, math, random, pickle, time
import numpy as np
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler,EarlyStopping
from keras.models import Sequential, Model, load_model
from keras.layers.pooling import AveragePooling1D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation, Input, merge, Conv2D, Reshape, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
import keras
from sklearn import preprocessing
from keras import backend as K
from keras import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import CMAPSAuxFunctions

# Constants

In [14]:
FeatureN = 14
nb_epoch = 250
batch_size = 512
FilterN = 10
FilterL = 10
rmse,sco,tm = [], [], []

ConstRUL = 125
TW = 30
Dataset = '1'

# Reshape data to fit a convNet

In [15]:
'''samples = np.reshape(X_train, newshape=(X_train.shape[0], int(X_train.shape[1]/nFeatures), nFeatures))
samplet = np.reshape(X_test, newshape=(X_test.shape[0], int(X_test.shape[1]/nFeatures), nFeatures))
targets = y_train
labelt = y_test'''

samples, targets, samplet, labelt = CMAPSAuxFunctions.get_X_y_from_Dataset('1', '../CMAPSSData/', constRUL, time_window)

print("Training data")
print(samples.shape)
print(targets.shape)
print("Testing data")
print(samplet.shape)
print(labelt.shape)
print("Training data")
print(samples[-5:,:])
print(targets[-5:])
print("Testing data")
print(samplet[-5:,:])
print(labelt[-5:])

Training data
(17731, 30, 14)
(17731,)
Testing data
(100, 30, 14)
(100,)
Training data
[[[ 0.42168675  0.12579028  0.17049291 ...  0.33333333 -0.27131783
   -0.07953604]
  [ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  ...
  [ 0.51204819  0.14453891  0.52464551 ...  0.         -0.62790698
   -0.34217067]
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]]

 [[ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  [-0.09638554 -0.02419882  0.53376097 ...  0.         -0.24031008
   -0.2761668 ]
  ...
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]
  

# Keras model

CNN model. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [16]:
def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Fit the keras model

Fit the Keras model to the data and determine its performance.

In [17]:
opt = Adam(lr=0, beta_1=0.5)
#DCNN = Model([input_layer], [y])
DCNN = RULCNNModel(TW, FeatureN)
#DCNN.compile(loss=get_score,optimizer=opt)
DCNN.compile(loss='mean_squared_error',optimizer=opt)
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)


startTime = time.clock()
history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
                   validation_data=(samplet, labelt), callbacks=[lrate])
endTime = time.clock()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), padding="same", name="C1", kernel_initializer="glorot_normal", activation="tanh")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), padding="same", name="C2", kernel_initializer="glorot_normal", activation="tanh")`
  import sys
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), padding="same", name="C3", kernel_initializer="glorot_normal", activation="tanh")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), padding="same", name="C4", kernel_initializer="gloro

Train on 17731 samples, validate on 100 samples
Epoch 1/250
17731/17731 [==============================] - 1s 70us/step - loss: 6498.8944 - val_loss: 4993.4146
Epoch 2/250
17731/17731 [==============================] - 0s 21us/step - loss: 5660.3737 - val_loss: 4571.3345
Epoch 3/250
17731/17731 [==============================] - 0s 22us/step - loss: 5215.3743 - val_loss: 4199.5576
Epoch 4/250
17731/17731 [==============================] - 0s 21us/step - loss: 4815.2465 - val_loss: 3860.5178
Epoch 5/250
17731/17731 [==============================] - 0s 22us/step - loss: 4448.4811 - val_loss: 3549.6030
Epoch 6/250
17731/17731 [==============================] - 0s 21us/step - loss: 4110.1514 - val_loss: 3260.2922
Epoch 7/250
17731/17731 [==============================] - 0s 22us/step - loss: 3797.6195 - val_loss: 2995.1016
Epoch 8/250
17731/17731 [==============================] - 0s 21us/step - loss: 3507.4581 - val_loss: 2745.6318
Epoch 9/250
17731/17731 [==============================]

17731/17731 [==============================] - 0s 21us/step - loss: 176.4628 - val_loss: 177.9437
Epoch 75/250
17731/17731 [==============================] - 0s 22us/step - loss: 177.5512 - val_loss: 163.5049
Epoch 76/250
17731/17731 [==============================] - 0s 22us/step - loss: 175.3709 - val_loss: 155.9617
Epoch 77/250
17731/17731 [==============================] - 0s 22us/step - loss: 176.9591 - val_loss: 187.3227
Epoch 78/250
17731/17731 [==============================] - 0s 22us/step - loss: 181.9026 - val_loss: 161.3189
Epoch 79/250
17731/17731 [==============================] - 0s 21us/step - loss: 170.8182 - val_loss: 161.4221
Epoch 80/250
17731/17731 [==============================] - 0s 21us/step - loss: 175.7751 - val_loss: 157.8793
Epoch 81/250
17731/17731 [==============================] - 0s 22us/step - loss: 170.5102 - val_loss: 197.4972
Epoch 82/250
17731/17731 [==============================] - 0s 22us/step - loss: 168.7735 - val_loss: 215.8596
Epoch 83/250
1

17731/17731 [==============================] - 0s 22us/step - loss: 118.0474 - val_loss: 176.6487
Epoch 221/250
17731/17731 [==============================] - 0s 22us/step - loss: 116.8628 - val_loss: 183.2626
Epoch 222/250
17731/17731 [==============================] - 0s 22us/step - loss: 117.1314 - val_loss: 188.4823
Epoch 223/250
17731/17731 [==============================] - 0s 22us/step - loss: 117.4579 - val_loss: 176.6675
Epoch 224/250
17731/17731 [==============================] - 0s 22us/step - loss: 116.3495 - val_loss: 179.1382
Epoch 225/250
17731/17731 [==============================] - 0s 21us/step - loss: 115.7039 - val_loss: 172.0908
Epoch 226/250
17731/17731 [==============================] - 0s 22us/step - loss: 116.7163 - val_loss: 181.9718
Epoch 227/250
17731/17731 [==============================] - 0s 22us/step - loss: 116.7217 - val_loss: 181.0440
Epoch 228/250
17731/17731 [==============================] - 0s 22us/step - loss: 115.2869 - val_loss: 181.9102
Epoch 

In [18]:
#Evaluate the model
score = DCNN.evaluate(samplet, labelt)
y_pred = DCNN.predict(samplet)
healtScore = CMAPSAuxFunctions.compute_health_score(labelt, y_pred)

print("Root Square Mean Error score: {}".format(np.sqrt(score)))
print("Health score: {}".format(healtScore))
print("Elapsed time: {}".format(endTime - startTime))

100/100 [==============================] - 0s 211us/step
Root Square Mean Error score: 13.1631671269127
Health score: [296.13544]
Elapsed time: 97.5915960797374


In [ ]:
engineUnit = 21

X_test2, _ = CMAPSAuxFunctions.retrieve_and_reshape_data(data_file_test, selected_features, time_window, 'train', unit_Number=engineUnit)
X_test2 = min_max_scaler.fit_transform(X_test2)

samplet2 = np.reshape(X_test2, newshape=(X_test2.shape[0], int(X_test2.shape[1]/nFeatures), nFeatures))

nnPred = modelRUL.predict(X_test2)
cnnPred = DCNN.predict(samplet2)

maxCycle = X_test2.shape[0]
faultCycle = y_test[engineUnit-1]
cycles = np.arange(maxCycle)
rulArray = np.arange(faultCycle, maxCycle+faultCycle)
rulArray[rulArray > constRUL] = constRUL
rulArray = np.flipud(rulArray)

#print(cycles)
#print(rulArray)

'''print("Testing data")
print(X_test2.shape)
print(X_test2[-5:,:])
print(nnPred)
print(cnnPred)'''

plotRUL(cycles, rulArray, nnPred, cnnPred, engineUnit)

In [24]:
def plotRUL(cycles, rulArray, nnPred, cnnPred, engineUnit):
    
    plt.clf()
    plt.plot(cycles, rulArray, 'bo-', label='RUL')
    plt.plot(cycles, nnPred, 'go-', label='NN Pred')
    plt.plot(cycles, cnnPred, 'ro-', label='CNN Pred')
    plt.legend()
    plt.xlabel("Time (Cycle)")
    plt.ylabel("RUL")
    plt.title("Test Engine Unit #{}".format(engineUnit))
    plt.show()